<a href="https://colab.research.google.com/github/rymarinelli/EHR/blob/main/EHR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql
!pip install pandas 
!pip install plotly
!pip install numba
!pip install statsmodels 
!pip install pytest

import pandasql as ps
import pandas as pd
import plotly.express as px
import numpy as np 
import logging
from collections import namedtuple
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go


from datetime import datetime

#Creating Log File and Formatting 
logging.basicConfig(filename='EHR.log', level=logging.DEBUG)
logging.Formatter("%(asctime)s;%(levelname)s;%(message)s")


try:
  import statsmodels.api as sm
  import statsmodels.formula.api as smf
  from statsmodels.tsa.ar_model import AutoReg
  

#
except ImportError:
  import os
  !pip install statsmodels --upgrade
  logging.error('Please restart runtime. Statsmodels may need to be updated')
  os.kill(os.getpid(), 9)
  

In [ ]:
class FileNotAdded(Exception):
  """
  Exception to notify that a file has not been uploaded to Github for the load_data function 
  """
  def __init__(self, message):            
        super().__init__(message)     

In [ ]:

def load_data(file):
  """ Util function to read in data from curl request using pandas API 

  Keyword arguments:
  file -- name of the name of the csv that was converted and is stored in github to be read

  Return:
  Pandas dataframe that may contain casting into types conductive to this analysis

  """
  try:
    if(file == "encounters"):
      df = pd.read_parquet(f"https://github.com/rymarinelli/EHR/blob/main/data/{file}.parquet?raw=true")
      df.CODE = df.CODE.astype('str')
      return df

    elif(file == "patients"):
      df = pd.read_parquet(f"https://github.com/rymarinelli/EHR/blob/main/data/{file}.parquet?raw=true")
      df.Id = df.Id.astype('str')
      return df
    
    elif(file == "observations"):
      df = pd.read_parquet(f"https://github.com/rymarinelli/EHR/blob/main/data/{file}.parquet?raw=true")
      df['DATE'] = pd.to_datetime(df['DATE'])
      df = df[df['DESCRIPTION'] == "Total score [MMSE]"].sort_values(by='DATE')
      df['YEAR'] = pd.DatetimeIndex(df['DATE']).year
      df.VALUE = df.VALUE.astype("float")
      return df

    #Files that do not require additional pre-processing are the default case here 
    else:
      return pd.read_parquet(f"https://github.com/rymarinelli/EHR/blob/main/data/{file}.parquet?raw=true")
  except:
      raise(FileNotAdded("The file selectied has not been added to the database yet.\n Please keep select to the following: \n 1.conditions \n 2.encounters \n 3.medications \n 4.observations \n 5. patients"))
  
def convert_file(file):
  """ Util function to convert data to parquet format 

  Keyword argument --
  Takes a csv file in working directory and convert to parquet 
  """
  df = pd.read_csv(f"{file}.csv")

  #gzip parameter further compresses parquet file. Other option may speed up compression at less effect ration
  df.to_parquet(f"{file}.parquet", compression = "gzip")

def clean_NA(df, col_name):
  """Util function to remove NA based on row values
     if a row value is NA based on a specific column
     the row is filtered out

 Keyword argument -- 
 df: Pandas dataframe 
 col_name = column in the pandas dataframe that is used for inclusion critera 
 """

  df['temp_col'] = pd.Series.isna(df[f"{col_name}"])
  df = df[df['temp_col'] == False]
  df = df.drop(['temp_col'], axis=1)
  return df

In [ ]:
conditions = load_data("conditions")
encounters = load_data("encounters")
patients   = load_data("patients")


#Question 1 


In [ ]:
alzheimer_df = conditions[ (conditions['DESCRIPTION'] == "Familial Alzheimer's disease of early onset (disorder)") | (conditions['DESCRIPTION'] == "Alzheimer's disease (disorder)")][['PATIENT', 'DESCRIPTION', "CODE"]].drop_duplicates().groupby(['DESCRIPTION', 'CODE']).count()

In [ ]:
logging.info(f'Question 1: {alzheimer_df.iloc}')

#Question 2

In [ ]:
#SELECTs encounter, code, description and gets a count of unique patients and divides the number of encounter by the unique patients
average_encounter_df = ps.sqldf( ''' SELECT  ENCOUNTERCLASS, CODE, DESCRIPTION, Count(ENCOUNTERCLASS) AS Number_Of_Encounters, Count(DISTINCT PATIENT) AS Patient_Count,  CAST(Count(ENCOUNTERCLASS) as FLOAT)/Count(DISTINCT PATIENT) AS Mean_Count 
          FROM encounters
          GROUP BY ENCOUNTERCLASS, CODE ''')

logging.info(f"Question 2: {average_encounter_df}")

#Question 3

In [ ]:
#Converts conditions column to time filters by Alzheimer's Disease in the Description field
#Sorts DataFrame by Start year 
#Groups by ID associated with each year

In [ ]:
conditions['START'] = pd.to_datetime(conditions['START'])
conditions = conditions[ ( conditions['DESCRIPTION'] == "Alzheimer's disease (disorder)")].sort_values(by='START').groupby("PATIENT").first()

In [ ]:
#Finds the difference in number of days and converts to year 
# The conditions table is already sorted to get the first diagnosis 
age = ps.sqldf('''
          SELECT conditions.PATIENT, conditions.START, patients.Birthdate, CAST((julianday(start) - julianday(BIRTHDATE))/365.25 as Int) as Age 
          FROM  conditions
          JOIN patients
          ON conditions.PATIENT = patients.Id
          ''')

In [ ]:
#Leftward most numbers are the labels for ages
#rightward most are the frequency of ages 
age = pd.DataFrame(age['Age'].value_counts())


In [ ]:
#Fixing Labels to the correct order
age = age.rename(columns={"Age": "Frequency"})
age['Age'] = age.index
logging.info(f"Question 3: {age}")

In [ ]:
fig = px.bar(age, x='Age', y="Frequency")
fig.show()
#Right skew mean is somewhere around 78

#Question 4

In [ ]:
#Using namedTuple collection as it supports the object dot syntax to make accessing the data more clear 
age_statistics = namedtuple("age_statistics", "mean std")
age_statistics  = age_statistics(np.array(ps.sqldf('''
          SELECT conditions.PATIENT, conditions.START, patients.Birthdate, CAST((julianday(start) - julianday(BIRTHDATE))/365.25 as Int) as Age 
          FROM  conditions
          JOIN patients
          ON conditions.PATIENT = patients.Id
          ''')['Age']).mean(), np.array(ps.sqldf('''
          SELECT conditions.PATIENT, conditions.START, patients.Birthdate, CAST((julianday(start) - julianday(BIRTHDATE))/365.25 as Int) as Age 
          FROM  conditions
          JOIN patients
          ON conditions.PATIENT = patients.Id
          ''')['Age']).std())
logging.info(f'Question 4 {age_statistics.std}')
logging.info(f'Question 4 {age_statistics.mean}')

In [ ]:
age_statistics.std
age_statistics.mean

77.0775075987842

#Question 5

In [ ]:
medications = load_data("medications")
conditions = load_data("conditions")
conditions = conditions[ ( conditions['DESCRIPTION'] == "Alzheimer's disease (disorder)")].drop_duplicates()

In [ ]:
#Finds the intersection of ids that are present in both the conditions tables and medications table 
len(set(conditions["PATIENT"]) & set(medications['PATIENT'])) 


611

#Question 6


In [ ]:
observations = load_data("observations")

In [ ]:
observation_plot = ps.sqldf(''' SELECT YEAR, CAST(AVG(VALUE) AS FLOAT )AS VALUE
         FROM observations
         GROUP BY YEAR
         ORDER BY YEAR 
         '''
)

logging.info(f"Question 6: {observation_plot}")

In [ ]:
fig = px.line(data_frame = observation_plot, x = "YEAR", y = "VALUE")
fig.show()

In [ ]:
type(observation_plot['VALUE'])

pandas.core.series.Series

In [ ]:
px.scatter(data_frame = observation_plot, x = "YEAR", y = "VALUE", trendline="lowess")

#Question 7

In [ ]:
conditions = load_data("conditions")
conditions = conditions[ ( conditions['DESCRIPTION'] == "Alzheimer's disease (disorder)")].sort_values(by='START').groupby("PATIENT").first()
conditions['YEAR'] = pd.DatetimeIndex(conditions['START']).year

In [ ]:
# Use CASE-WHEN to make a dummy vvariable to split by 
age_comparison = ps.sqldf('''
          SELECT conditions.PATIENT, conditions.START, patients.Birthdate, CAST((julianday(start) - julianday(BIRTHDATE))/365.25 as Int) as Age, 
          CASE 
          WHEN CAST((julianday(start) - julianday(BIRTHDATE))/365.25 as Int) >= 75 THEN 1
          WHEN CAST((julianday(start) - julianday(BIRTHDATE))/365.25 as Int) < 75 THEN 0
          END as Age_Split
          FROM  conditions
          JOIN patients
          ON conditions.PATIENT = patients.Id
          ''')

In [ ]:
#Taking the average to create visualizations, the indiviudal level is too noisy to make good insights 
age_viz = ps.sqldf("""
            SELECT AVG(observations.VALUE) AS VALUE, observations.YEAR, CAST(age_comparison.Age_Split AS Integer) AS Age_Split, age_comparison.Age
            FROM observations
            JOIN age_comparison 
            ON age_comparison.PATIENT = observations.PATIENT
            GROUP BY YEAR
          """)

In [ ]:
#When Age_Split = 0, they are under 75
#When Age_split = 1, they are 75 or over 
fig = px.scatter(age_viz, x="YEAR", y="VALUE", title='Age of AD Diagnosis', facet_col= "Age_Split", trendline="lowess", color="Age_Split")
fig.show()

#Question 8

In [ ]:
observations.groupby(['YEAR', 'VALUE'])

In [ ]:
observations_AR = observations.groupby('YEAR').agg({'VALUE': 'mean'})

In [ ]:
X = pd.DataFrame(observations_AR.index)
Y = observations_AR['VALUE']
reg = LinearRegression(n_jobs=-1).fit(X, Y)
print(reg.coef_)
logging.info(f"Question 8: {reg.coef_}")

[-0.01283733]


In [ ]:
results = AutoReg(observations['VALUE'], lags = [1, 5, 10]).fit()
results = pd.concat([pd.DataFrame(results.params), pd.DataFrame(results.pvalues)], axis=1)
results.columns = ['Beta_Coefficients', 'P_Value']

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:579: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/ar_model.py:252: FutureWarning:

The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 



In [ ]:

fig = go.Figure(data=[go.Table(
    header=dict(values=list(results.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[results.Beta_Coefficients, results.P_Value],
               fill_color='lavender',
               align='left'))
])

fig.show()

In [ ]:
patients['Gender_Indicator'] = pd.get_dummies(patients['GENDER'])['M']
age_comparison['Age'] = np.subtract(np.array(age_comparison['Age']), (np.array(age_comparison['Age']).mean()))

In [ ]:
model_df = ps.sqldf('''
          SELECT patients.Id, patients.Gender_Indicator, 
          age_comparison.Age, age_comparison.Age_Split, 
          observations.VALUE, observations.YEAR
          FROM patients
          JOIN age_comparison
          ON patients.Id = age_comparison.PATIENT
          JOIN observations 
          ON observations.PATIENT = patients.Id
          '''
          )

In [ ]:
y = model_df['VALUE']
X = model_df[['Gender_Indicator','Age', 'Age_Split']]
X.set_index(model_df['Id'])

,Gender_Indicator,Age,Age_Split
Id,,,
bc22dfdf-86ba-bbbd-6e46-1852f6bc5e25,0,-3.077508,0
bc22dfdf-86ba-bbbd-6e46-1852f6bc5e25,0,-3.077508,0
bc22dfdf-86ba-bbbd-6e46-1852f6bc5e25,0,-3.077508,0
9d9763d8-124d-e1e5-0c3a-ff571492b286,0,8.922492,1
9d9763d8-124d-e1e5-0c3a-ff571492b286,0,8.922492,1
...,...,...,...
592d65f0-4ea0-593b-b795-18083e639096,1,5.922492,1
f84aae1b-9884-a58b-753f-4c644fa27821,0,-2.077508,1
f84aae1b-9884-a58b-753f-4c644fa27821,0,-2.077508,1


In [ ]:
X = sm.add_constant(X)
mod = sm.OLS(y, X)
res = mod.fit()

print(res.summary())
logging.info(f"Question 8: {res.summary}")

                            OLS Regression Results                            
Dep. Variable:                  VALUE   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3700
Date:                Thu, 05 Aug 2021   Prob (F-statistic):              0.775
Time:                        16:49:14   Log-Likelihood:                -7449.1
No. Observations:                2214   AIC:                         1.491e+04
Df Residuals:                    2210   BIC:                         1.493e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               12.6796      0.375  

In [ ]:
mod = smf.ols(formula='VALUE ~ Age + Gender_Indicator + Age_Split + Age_Split*Gender_Indicator', data=model_df)
res = mod.fit()
logging.info(f"Question 8: {res.summary}")
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  VALUE   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2816
Date:                Thu, 05 Aug 2021   Prob (F-statistic):              0.890
Time:                        16:49:14   Log-Likelihood:                -7449.1
No. Observations:                2214   AIC:                         1.491e+04
Df Residuals:                    2209   BIC:                         1.494e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

#Question 9

For the initial fit of with year as the dependent variable, there is a slight negative association. For each year, there is a - 0.12 drop in the MMSE score. This is a logical progression of aging. Since a higher MMSE suggests healthier cognitive functions, the erosion of  mental acuity is expected with age.  In addition to simple regression, an ARIMA model is also applied to observe lag effects. The effects of aging from five years ago are still affecting the MMSE score. The effects of aging from ten years also have some effect on the present. The associated p-value is over .05 threshold, but it is likely still contributing to the overall negative trend. 
 
For the  multivariate regression model, a per unit change in age is associated with a .04 decrease in MMSE. The baseline for the model has a MMSE of 12.67; this is the intercept term. As for gender, men have a lower MMSE of women. Holding all other factors equal, gender is associated with a decrease of .04. Essentially, being a man has the same effect of being a woman but being a year older. 


#Question 10 

When looking to answer this hypothesis, a dummy variable is encoded based on when someone had their AD diagnosis. If they were 75 or older at their diagnosis, it is encoded as 1. If not, the variable is encoded as zero. Holding all other factors equal, a person that had their diagnosis at or after reaching 75, should have a half a point higher than a person that had their diagnosis earlier. This is likely detecting when patients started their cognitive declines. A person with an earlier onset has a longer time to degrade and get a lower MMSE. If you are older, you have less time to degrade and are likely to pass away from other ailments before your MMSE can be measured. Effectively, people with later onsets might just too soon before their MMSE can degrade. 